# GOMC Example for the Gibbs Ensemble (GEMC) using MoSDeF [1, 2, 5-10, 13-17]

### Note: In this specific example, we will be using the GEMC_NVT ensemble.

## Import the required packages and specify the force field (FF) being used. 

## Note: For GOMC, the residue names are treated as molecules, so the residue names must be unique for each different molecule. [1, 2, 13-17]

## Note: Each residue can be set to a different FF, which is done by setting the residue name to a FF in a dictionary (FF_Dict).  The FF selection can be a FF name (set from foyer FF repositor) or a specified FF xml file. [1, 2, 13-17]



In [11]:
"""import mbuild as mb
from foyer import Forcefield
import mbuild.formats.charmm_writer as mf_charmm
import mbuild.formats.gomc_conf_writer as gomc_control


FF_file_water = '../common/spce.xml'
water = mb.load('O', smiles=True)
water.name = 'H2O'

water.energy_minimize(forcefield=FF_file_water, steps=10**5)

FF_file_ethanol = 'oplsaa'
ethanol = mb.load('CCO', smiles=True)
ethanol.name = 'ETO'
ethanol.energy_minimize(forcefield=FF_file_ethanol, steps=10**5)

FF_dict = {water.name: FF_file_water, ethanol.name: FF_file_ethanol}

residues_list = [ethanol.name, water.name]

fix_bonds_angles_residues = [water.name]"""

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"import mbuild as mb\nfrom foyer import Forcefield\nimport mbuild.formats.charmm_writer as mf_charmm\nimport mbuild.formats.gomc_conf_writer as gomc_control\n\n\nFF_file_water = '../common/spce.xml'\nwater = mb.load('O', smiles=True)\nwater.name = 'H2O'\n\nwater.energy_minimize(forcefield=FF_file_water, steps=10**5)\n\nFF_file_ethanol = 'oplsaa'\nethanol = mb.load('CCO', smiles=True)\nethanol.name = 'ETO'\nethanol.energy_minimize(forcefield=FF_file_ethanol, steps=10**5)\n\nFF_dict = {water.name: FF_file_water, ethanol.name: FF_file_ethanol}\n\nresidues_list = [ethanol.name, water.name]\n\nfix_bonds_angles_residues = [water.name]"

## Build the main simulation liquid box (box 0) and the vapor (box 1) for the simulation [1, 2, 13-17]

In [3]:
water_ethanol_box_liq = mb.fill_box(compound=[water, ethanol],
                                    density= 950,
                                    compound_ratio=[0.8, 0.2] ,
                                    box=[3.0, 3.0, 3.0])

water_ethanol_box_vap = mb.fill_box(compound=[water,ethanol],
                                    density= 100,
                                    compound_ratio=[0.8, 0.2],
                                    box=[8, 8, 8])

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'water' is not defined

## Build the Charmm object, which is required to write the FF (.inp), psf, pdb, and GOMC control files [1, 2, 5-10, 13-17]

## The reorder_res_in_pdb_psf command reorders the psf and pdb to the order residues variable (i.e., the residues_list in this case) [1, 2, 13-17].  

## The fix_res_bonds_angles command fixes the angles and bonds for water in the Charmm FF file.  Note: This is specific to GOMC, as it sets the bond and angle k-values to 999999999999 [1, 2, 5-10, 13-17].

In [13]:
charmm = mf_charmm.Charmm(water_ethanol_box_liq,
                          'GEMC_NVT_water_ethanol_liq',
                          structure_box_1=water_ethanol_box_vap,
                          filename_box_1='GEMC_NVT_water_ethanol_vap',
                          ff_filename="GEMC_NVT_water_ethanol_FF",
                          forcefield_selection=FF_dict,
                          residues=residues_list,
                          bead_to_atom_name_dict=None,
                          fix_residue=None,
                          gomc_fix_bonds_angles=fix_bonds_angles_residues,
                          reorder_res_in_pdb_psf=True
                          )

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'charmm = mf_charmm.Charmm(water_ethanol_box_liq,\n                          \'GEMC_NVT_water_ethanol_liq\',\n                          structure_box_1=water_ethanol_box_vap,\n                          filename_box_1=\'GEMC_NVT_water_ethanol_vap\',\n                          ff_filename="GEMC_NVT_water_ethanol_FF",\n                          forcefield_selection=FF_dict,\n                          residues=residues_list,\n                          bead_to_atom_name_dict=None,\n                          fix_residue=None,\n                          gomc_fix_bonds_angles=fix_bonds_angles_residues,\n                          reorder_res_in_pdb_psf=True\n                          )'

## Write the write the FF (.inp), psf, pdb, and GOMC control files [1, 2, 5-10, 13-17]

In [17]:
charmm.write_inp()

charmm.write_psf()

charmm.write_pdb()


gomc_control.write_gomc_control_file(charmm, 'in_GEMC_NVT.conf', 'GEMC_NVT', 100, 300,
                                     input_variables_dict={"VDWGeometricSigma": True,
                                                           "Rcut": 12,
                                                           "RcutCoulomb_box_1": 20,
                                                           "DisFreq": 0.20,
                                                           "RotFreq": 0.20, 
                                                           "IntraSwapFreq": 0.10,
                                                           "SwapFreq": 0.20,
                                                           "RegrowthFreq": 0.20,
                                                           "CrankShaftFreq": 0.09,
                                                           "VolFreq": 0.00,
                                                           "MultiParticleFreq": 0.01,
                                                           }
                                    )

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


******************************

The charmm force field file writer (the write_inp function) is running
******************************

The charmm force field file writer (the write_inp function) is running
******************************

writing the GOMC force field file 
No urey bradley terms detected, will use angle_style harmonic
will use CHARMM_torsions  =  K0 + K1 * (1 + Cos[n1*(t) - (d1)] ) + K2 * (1 + Cos[n2*(t) - (d2)] ) + K3 * (1 + Cos[n3*(t) - (d3)] ) + K4 * (1 + Cos[n4*(t) - (d4)] ) + K5 * (1 + Cos[n5*(t) - (d5)] ) 
! RB-torsion to CHARMM dihedral conversion error is OK [error <= 10^(-10)]
! Maximum( |(RB-torsion calc)-(CHARMM dihedral calc)| ) =  1.9984014443252818e-15

NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
self.non_bonded_type = LJ
forcefield_dict = {2: 'opls_135_ETO', 6: 'opls_157_ETO', 4: 'opls_154_ETO', 3: 'opls_140_ETO', 5: 'opls_155_ETO', 1: 'o_spce_H2O', 0: 'h_spce_H2O'}
******************************

The charmm X-plor format psf 

'GOMC_CONTROL_FILE_WRITTEN'